# Understanding MetadataSync mappings for text columns (Synapse Spark ↔ Serverless SQL)

In this demo, we’ll explore how **MetadataSync** maps **String** and **Varchar** columns between **Synapse Spark** and **Synapse Serverless SQL pool**.

---

## Objective

By the end of this demo, you’ll understand:

- How text columns in Synapse Spark are mapped by MetadataSync to **Serverless SQL pool**.
- The difference in behavior when using **Delta** vs **Parquet** tables.
- Why certain mappings (like `VARCHAR(8000)` or `VARCHAR(MAX)`) are **by design**.
- How to work around performance limitations caused by large text types.

In [3]:
# ---------------------------
# Create database (if not exists)
# ---------------------------
spark.sql(f"CREATE DATABASE IF NOT EXISTS string_column_demo")

StatementMeta(sparkplay01, 4, 4, Finished, Available, Finished)

DataFrame[]

## Step 1 - Create a Delta table in Spark

Let’s start by creating a simple **Delta table** with three columns:  
`id`, `event_time`, and `description`.

Notice that the `description` column is defined as **String**.

In [4]:
%%sql
-- Switch to the target database
USE string_column_demo;

-- Create a Delta table with 3 columns
CREATE TABLE IF NOT EXISTS t1_delta_string (
    id INT,
    event_time TIMESTAMP,
    description STRING
)
USING DELTA;

-- Insert one record
INSERT INTO t1_delta_string VALUES (1, current_timestamp(), 'She sells seashells by the seashore');

StatementMeta(sparkplay01, 4, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### Observation

- In **Serverless SQL pool**, there is **no “String” data type**.  
- MetadataSync must choose an equivalent SQL type — in this case, it will use **VARCHAR(8000)**.

You can confirm this by checking the table schema on Serverless SQL.

This behavior is **by design** and consistent across **Synapse** and **Microsoft Fabric**.

## Step 2 - Testing with a VARCHAR(n) column

Let’s see what happens when we explicitly define the column as `VARCHAR(40)` in Spark instead of `String`.

MetadataSync will map the `VARCHAR(40)` to `VARCHAR(160)` in Serverless SQL pool — that’s because it multiplies the size by 4.


✅ **Summary**
| Spark Type | Serverless Type | Notes |
|-------------|-----------------|-------|
| `String` | `VARCHAR(8000)` | Default mapping |
| `VARCHAR(40)` | `VARCHAR(160)` | Multiplied by 4 |


In [5]:
%%sql
-- Switch to the target database
USE string_column_demo;

-- Create a Delta table with 3 columns
CREATE TABLE IF NOT EXISTS t1_delta_varchar_40 (
    id INT,
    event_time TIMESTAMP,
    description VARCHAR(40)
)
USING DELTA;

-- Insert one record
INSERT INTO t1_delta_varchar_40 VALUES (1, current_timestamp(), 'She sells seashells by the seashore');

StatementMeta(sparkplay01, 4, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Step 3 - Switching to Parquet Tables

Now, let’s test the same logic, but this time creating **Parquet** tables instead of Delta.

In Parquet, you’ll notice that **regardless of using `String` or `VARCHAR(n)`**, MetadataSync will **always map** the column to **`VARCHAR(MAX)`** in Serverless SQL pool.


In [6]:
%%sql
-- Switch to the target database
USE string_column_demo;

-- Create a Parquet table with 3 columns
CREATE TABLE IF NOT EXISTS t1_parquet_string (
    id INT,
    event_time TIMESTAMP,
    description STRING
)
USING PARQUET;

-- Insert one record
INSERT INTO t1_parquet_string VALUES (1, current_timestamp(), 'She sells seashells by the seashore');

StatementMeta(sparkplay01, 4, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
-- Switch to the target database
USE string_column_demo;

-- Create a Parquet table with 3 columns
CREATE TABLE IF NOT EXISTS t1_parquet_varchar_40 (
    id INT,
    event_time TIMESTAMP,
    description VARCHAR(40)
)
USING PARQUET;

-- Insert one record
INSERT INTO t1_parquet_varchar_40 VALUES (1, current_timestamp(), 'She sells seashells by the seashore');

StatementMeta(sparkplay01, 4, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Step 4 - Creating Statistics error (Serverless SQL pool)

Now switch to **Synapse Serverless SQL pool**.

Let’s try to create **statistics** on the `description` column that was mapped as `VARCHAR(MAX)`.

Run the following command in **SSMS** or **Synapse Studio (SQL On-Demand)**:

```sql
CREATE STATISTICS sDescription ON t1_parquet_varchar_40([description])
```



You’ll see an error similar to:

```
Msg 2729, Level 16, State 1
Column 'description' in table 't1_parquet' cannot be used in a statistics definition because its data type is a large object (LOB).

```

Explanation:

- Columns of type VARCHAR(MAX) are considered Large Objects (LOBs).
- The Serverless SQL engine doesn’t support creating statistics on LOB columns.
- Without statistics, the query optimizer has limited visibility into data distribution, which may cause suboptimal query plans.

## Step 5 - Creating a SQL View (Serverless SQL pool)

We’ll stay on the **Serverless SQL pool** side for this step.

Since you can’t create statistics on columns defined as `VARCHAR(MAX)`,  a good workaround is to create a **custom view** that reads the same Parquet files using the `OPENROWSET` function —  but this time explicitly defining the column data types and lengths in the `WITH` clause.

This approach lets you:

- Control the data types manually (for example, use `VARCHAR(40)` instead of `VARCHAR(MAX)`).
- Allow **statistics creation** on those columns.
- Improve **query performance**.

Example:

```sql
CREATE SCHEMA sch1
GO
CREATE VIEW sch1.v1_parquet_varchar_40
AS
SELECT
    *
FROM
    OPENROWSET(
        BULK 'https://<storage-path>.dfs.core.windows.net/<container-name>/synapse/workspaces/<workspace-name>/warehouse/<db-name>.db/<table-name>/**',
        FORMAT = 'PARQUET'
    )
    WITH (
        id INT,
        event_time DATETIME,
        description VARCHAR(40)
    ) AS [result];
GO

SELECT * FROM sch1.v1_parquet_varchar_40
```


## Step 6 - Challenge

Alright, now for the **challenge**!  
This exercise ties together everything we discussed in this notebook.

Let’s go to the next cell that will create a **Delta table** that includes a column of type **String**.  
It will also insert a single record containing a string with **10,000 characters**.

Once you’ve executed it, switch to **Synapse Serverless SQL pool** and try to **SELECT** this new table.

You’ll notice that the query **fails with an error**.

> **Your mission:**
>
> 1. Identify the **root cause** of this error.  
> 2. Propose a **solution or workaround** to fix it.  
> 3. (Bonus) Explain *why* this behavior happens when using certain data types across Spark and Serverless.

In [8]:
%%sql
-- Switch to the target database
USE string_column_demo;

-- Create a Delta table with 3 columns
CREATE TABLE IF NOT EXISTS t1_delta_varchar_string_challenge (
    id INT,
    event_time TIMESTAMP,
    description STRING
)
USING DELTA;

-- Insert one record with a string with 10K caracters
INSERT INTO t1_delta_varchar_string_challenge 
VALUES (1, current_timestamp(), repeat('A', 10000));

StatementMeta(sparkplay01, 4, 19, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

That’s the end of the demo — great job if you made it this far! 👏  
Feel free to DROP the `string_column_demo` database:

In [1]:
spark.sql("DROP DATABASE IF EXISTS string_column_demo CASCADE")

StatementMeta(sparkplay01, 4, 2, Finished, Available, Finished)

DataFrame[]